In [3]:
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta
import os
from tqdm import tqdm
import keyboard
import msgpack as mp
import msgpack_numpy as mpn
import base64
from datetime import datetime


In [2]:
# sub = input('subject: ')
# arm = input('arm: ')

In [20]:
print(f'Videos for {sub}: ')
fldr = 'controls' if sub[0]=='C' else 'patients'
vids = [x for x in os.listdir(f'{fldr}/{sub}') if x.find('.avi')!=-1]
print(vids)

Videos for C5: 
['C5_11032020_125023.avi', 'C5_11032020_170218.avi']


In [21]:
vid = vids[0]
frtime = pd.read_csv(f'{fldr}/{sub}/'+vid.split('.')[0]+'_time.csv', parse_dates=[0], index_col=0)
frtime.index.name = 'time'
imu = pd.read_csv(f'watch raw data/{arm}.csv', parse_dates=['time'], index_col='time')
imu = imu.between_time(frtime.index[0].time(), frtime.index[-1].time())
frtime = frtime.index

In [22]:
%matplotlib qt

In [23]:
cap = cv2.VideoCapture(f'{fldr}/{sub}/{vid}')
frames = []
  
while(True):
      
    # Capture frames in the video
    ret, frame = cap.read()
    if ret:
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    else:
        break
  
# release the cap object
cap.release()


In [24]:
imutime = [] 
for t in tqdm(frtime[:len(frames)-1]):
    imutime.append(imu.index[imu.index.get_loc(t, method='nearest')])
grps = []
for t in tqdm(imutime):
    grps.append(imu.between_time((t - timedelta(seconds=2)).time(), (t + timedelta(seconds=1)).time()))

100%|██████████| 10563/10563 [00:35<00:00, 300.45it/s]


In [25]:
print(arm)

left


In [27]:
start_at = 3500
i=0

for frame, t, dat in zip(frames[start_at+1:], imutime[start_at:], grps[start_at:]):
    if keyboard.is_pressed('q'):
        plt.close()
        break
    else:
        plt.clf()
        plt.subplot(121)
        plt.title(i)
        plt.imshow(frame)
        plt.subplot(322)
        plt.title(t)
        plt.plot(dat.ax)
        plt.axvline(t, color='r')
        plt.ylim(imu.ax.min(), imu.ay.max())
        plt.subplot(324)
        plt.plot(dat.ay)
        plt.axvline(t, color='r')
        plt.ylim(imu.ay.min(), imu.ay.max())
        plt.subplot(326)
        plt.plot(dat.az)
        plt.axvline(t, color='r')
        plt.ylim(imu.az.min(), imu.az.max())
        i+=1
        plt.pause(0.0001)